# Final Project

In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
!pip install folium
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors

     |████████████████████████████████| 94 kB 6.0 MB/s  eta 0:00:01


Use BeautifulSoup to scrap the list of postal codes from Wikipedia.

In [7]:
url="https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969"
source=requests.get(url).text
soup=BeautifulSoup(source, 'xml')
table=soup.find('table')
col_names=['PostalCode','Borough','Neighborhood']
df=pd.DataFrame(columns=col_names)

In [8]:
for tr_cell in table.find_all('tr'):
    data=[]
    for td_cell in tr_cell.find_all('td'):
        data.append(td_cell.text.strip())
    if len(data)==3:
        df.loc[len(df)]=data
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Clean data by removing 'Not assigned' entries and grouping neighborhoods with the same postal code.

In [9]:
df=df[df['Borough']!='Not assigned']
group=df.groupby('PostalCode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
group=group.reset_index(drop=False)
group.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)
df_new=pd.merge(df,group,on='PostalCode')
df_new.drop(['Neighborhood'],axis=1,inplace=True)
df_new.drop_duplicates(inplace=True)
df_new.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
df_new.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
df_new.shape

(103, 3)

Obtain geospatial data

In [11]:
geo_df=pd.read_csv('https://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge geospatial data to postal code data

In [12]:
geo_df.rename(columns={'Postal Code':'PostalCode'},inplace=True)
geo_merge=pd.merge(geo_df,df_new,on='PostalCode')
geo_data=geo_merge[['PostalCode','Borough','Neighborhood','Latitude','Longitude']]
geo_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Limit analysis to Downtown Toronto

In [13]:
toronto=geo_data[geo_data['Borough'].str.contains("Downtown Toronto")]
toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
51,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
53,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
54,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
55,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
56,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
58,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
59,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752


Call Foursquare API

In [14]:
CLIENT_ID='LK2N1ODWQTTAIUQXHTFPZZPZTNU0NQ05UH2FO3FXHRYOAYKW'
CLIENT_SECRET='DTENBIQUGHTNWMUF2GLJYY1T3S0GMGLTBPINSYEOEMYS4OCL'
VERSION='20180604'

In [15]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    limit=100
    venues_list=[]
    for name,lat,lon in zip(names, latitudes, longitudes):
        print(name)
        
        #create API request URL
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lon, 
            radius, 
            limit)
        
        #make GET request
        results=requests.get(url).json()["response"]['groups'][0]['items']
        
        #return relevant information on each nearby venue
        venues_list.append([(name,lat,lon,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category']
    
    return(nearby_venues)

Explore neighborhoods in Downtown Toronto

In [16]:
toronto_venues=getNearbyVenues(names=toronto['Neighborhood'],latitudes=toronto['Latitude'],longitudes=toronto['Longitude'])

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


In [17]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


In [18]:
venuecount=toronto_venues.groupby('Neighborhood').count()
venuecount['Venue']

Neighborhood
Berczy Park                                                                                                    59
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport     16
Central Bay Street                                                                                             64
Christie                                                                                                       16
Church and Wellesley                                                                                           70
Commerce Court, Victoria Hotel                                                                                100
First Canadian Place, Underground city                                                                        100
Garden District, Ryerson                                                                                      100
Harbourfront East, Union Station, Toronto Islands                          

One hot encoding

In [19]:
toronto_onehot=pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_group=toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Find Italian Restaurants

In [21]:
"Italian Restaurant" in toronto_venues['Venue Category'].unique()

True

In [24]:
len(toronto_group[toronto_group["Italian Restaurant"]>0])

13

In [27]:
toronto_it=toronto_group[["Neighborhood","Italian Restaurant"]]
toronto_it

,Neighborhood,Italian Restaurant
0,Berczy Park,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000
2,Central Bay Street,0.046875
3,Christie,0.062500
4,Church and Wellesley,0.014286
5,"Commerce Court, Victoria Hotel",0.040000
6,"First Canadian Place, Underground city",0.010000
7,"Garden District, Ryerson",0.020000
8,"Harbourfront East, Union Station, Toronto Islands",0.030000
9,"Kensington Market, Chinatown, Grange Park",0.000000


K-means Clustering

In [29]:
from sklearn.cluster import KMeans
tcluster=3
toronto_cluster=toronto_it.drop(["Neighborhood"],1)
kmeans=KMeans(n_clusters=tcluster,random_state=1)
kmeans.fit_transform(toronto_cluster)
kmeans.labels_[0:20]

array([0, 0, 1, 1, 0, 2, 0, 2, 2, 0, 2, 0, 0, 0, 2, 1, 2, 2, 1],
      dtype=int32)

In [38]:
toronto_merged=toronto_it.copy()
toronto_merged["Cluster Labels"]=kmeans.labels_
toronto_merged

,Neighborhood,Italian Restaurant,Cluster Labels
0,Berczy Park,0.000000,0
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0
2,Central Bay Street,0.046875,1
3,Christie,0.062500,1
4,Church and Wellesley,0.014286,0
5,"Commerce Court, Victoria Hotel",0.040000,2
6,"First Canadian Place, Underground city",0.010000,0
7,"Garden District, Ryerson",0.020000,2
8,"Harbourfront East, Union Station, Toronto Islands",0.030000,2
9,"Kensington Market, Chinatown, Grange Park",0.000000,0


In [39]:
toronto_merged=toronto_merged.join(toronto_venues.set_index("Neighborhood"),on="Neighborhood")
print(toronto_merged.shape)
toronto_merged.head()

(1215, 9)


,Neighborhood,Italian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.0,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.0,0,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.0,0,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.0,0,43.644771,-79.373306,Goose Island Brewhouse,43.647329,-79.373541,Beer Bar
0,Berczy Park,0.0,0,43.644771,-79.373306,Biff's Bistro,43.647085,-79.376342,French Restaurant


In [40]:
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged.head()

,Neighborhood,Italian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.00,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
6,"First Canadian Place, Underground city",0.01,0,43.648429,-79.382280,The National Club,43.649343,-79.380574,Wine Bar
6,"First Canadian Place, Underground city",0.01,0,43.648429,-79.382280,Petit Four Bakery,43.647744,-79.379588,Sandwich Place
6,"First Canadian Place, Underground city",0.01,0,43.648429,-79.382280,Soho House Toronto,43.648734,-79.386541,Speakeasy
6,"First Canadian Place, Underground city",0.01,0,43.648429,-79.382280,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant


Find coordinates of Toronto

In [42]:
address='Toronto, CA'
geolocator=Nominatim(user_agent="ny_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto are 43.6534817, -79.3839347.


Create map of clusters

In [46]:
map_clusters=folium.Map(location=[latitude,longitude],zoom_start=13)
markers_colors={}
markers_colors[0]='red'
markers_colors[1]='blue'
markers_colors[2]='green'
for lat,lon,cluster in zip(toronto_merged['Neighborhood Latitude'],toronto_merged['Neighborhood Longitude'],toronto_merged['Cluster Labels']):   
    folium.features.CircleMarker([lat, lon],radius=5,color=markers_colors[cluster],
        fill_color=markers_colors[cluster],fill_opacity=0.7).add_to(map_clusters)    
map_clusters

Cluster 0

In [47]:
toronto_merged.loc[(toronto_merged['Cluster Labels']==0)&(toronto_merged['Venue Category']=='Italian Restaurant')]

,Neighborhood,Italian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
6,"First Canadian Place, Underground city",0.010000,0,43.648429,-79.38228,Mercatto,43.650243,-79.38082,Italian Restaurant
4,Church and Wellesley,0.014286,0,43.665860,-79.38316,Dal Moro's Fresh Pasta To Go,43.666641,-79.38540,Italian Restaurant


Cluster 1

In [48]:
toronto_merged.loc[(toronto_merged['Cluster Labels']==1)&(toronto_merged['Venue Category']=='Italian Restaurant')]

,Neighborhood,Italian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
15,"St. James Town, Cabbagetown",0.048780,1,43.667967,-79.367675,Hey Lucy,43.664075,-79.368655,Italian Restaurant
15,"St. James Town, Cabbagetown",0.048780,1,43.667967,-79.367675,F'Amelia,43.667536,-79.368613,Italian Restaurant
18,"University of Toronto, Harbord",0.060606,1,43.662696,-79.400049,Daddyo's,43.664622,-79.402685,Italian Restaurant
18,"University of Toronto, Harbord",0.060606,1,43.662696,-79.400049,Piano Piano,43.662949,-79.402898,Italian Restaurant
2,Central Bay Street,0.046875,1,43.657952,-79.387383,Tosto,43.661198,-79.386414,Italian Restaurant
2,Central Bay Street,0.046875,1,43.657952,-79.387383,Mercatto,43.660391,-79.387664,Italian Restaurant
3,Christie,0.062500,1,43.669542,-79.422564,Vinny’s Panini,43.670679,-79.426148,Italian Restaurant
2,Central Bay Street,0.046875,1,43.657952,-79.387383,Scaddabush Italian Kitchen & Bar,43.658920,-79.382891,Italian Restaurant


Cluster 2

In [49]:
toronto_merged.loc[(toronto_merged['Cluster Labels']==2)&(toronto_merged['Venue Category']=='Italian Restaurant')]

,Neighborhood,Italian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
14,St. James Town,0.024390,2,43.651494,-79.375418,Mercatto,43.650243,-79.380820,Italian Restaurant
16,Stn A PO Boxes,0.030303,2,43.646435,-79.374846,Carisma,43.649617,-79.375434,Italian Restaurant
17,"Toronto Dominion Centre, Design Exchange",0.030000,2,43.647177,-79.381576,TOCA,43.645431,-79.387059,Italian Restaurant
17,"Toronto Dominion Centre, Design Exchange",0.030000,2,43.647177,-79.381576,Metropolitan Resto Bar,43.650062,-79.377181,Italian Restaurant
17,"Toronto Dominion Centre, Design Exchange",0.030000,2,43.647177,-79.381576,Mercatto,43.650243,-79.380820,Italian Restaurant
16,Stn A PO Boxes,0.030303,2,43.646435,-79.374846,Mirto,43.650702,-79.376307,Italian Restaurant
16,Stn A PO Boxes,0.030303,2,43.646435,-79.374846,Metropolitan Resto Bar,43.650062,-79.377181,Italian Restaurant
8,"Harbourfront East, Union Station, Toronto Islands",0.030000,2,43.640816,-79.381752,Piazza Manna,43.641460,-79.377719,Italian Restaurant
10,"Queen's Park, Ontario Provincial Government",0.032258,2,43.662301,-79.389494,Mercatto,43.660391,-79.387664,Italian Restaurant
7,"Garden District, Ryerson",0.020000,2,43.657162,-79.378937,Trattoria Mercatto,43.654453,-79.380974,Italian Restaurant


### Conclusion

There are few Italian restaurants in Cluster 0, so in neighborhoods like Berczy Park, CN Tower, Kensington Market, Regent Park, Harbourfront, Richmond and Rosedale, there is an oppurtunity to open this kind of business where there isn't one yet.